In [1]:
import pandas as pd 
dataset = pd.DataFrame(pd.read_csv("ORDER_INVENTORY.csv"))
dataset

<ipython-input-1-4476b9c01186>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FileNotFoundError: [Errno 2] No such file or directory: 'ORDER_INVENTORY.csv'

In [2]:
product_set=dataset.groupby("Order ID")["Item Name"].apply(set).to_dict()

In [3]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
List_order_id=list(filter_basket.keys())
List_product_id=[list(values) for values in filter_basket.values()]
dict1={"Order_ID":List_order_id,"Product_ID":List_product_id}
df1=pd.DataFrame(dict1)
print(df1)

           Order_ID                                         Product_ID
0      AE-2013-1130  [frozen vegetables, escalope, green tea, ham, ...
1      AE-2013-1530  [mushroom cream sauce, turkey, cottage cheese,...
2      AE-2014-2840  [fruit/vegetable juice, newspapers, baking pow...
3      AE-2014-4120                 [red/blush wine, other vegetables]
4      AG-2013-2840  [french fries, grated cheese, avocado, green tea]
...             ...                                                ...
12160  ZI-2014-6740  [beef, canned fish, brown bread, newspapers, c...
12161  ZI-2014-6750                        [salad, french fries, eggs]
12162  ZI-2014-7160      [meat, chicken, other vegetables, whole milk]
12163  ZI-2014-9540  [canned vegetables, whole milk, curd, soda, wh...
12164  ZI-2014-9550                            [soda, UHT-milk, sugar]

[12165 rows x 2 columns]


In [4]:
# Create a list of unique items
unique_items = sorted(set(item for sublist in df1['Product_ID'] for item in sublist))
 
# Initialize an empty dictionary to hold binary values for each item
binary_data = {item: [] for item in unique_items}
 
# Iterate over each row in the original DataFrame
for bought_together in df1['Product_ID']:
    # Initialize a binary row for the current transaction
    binary_row = {item: 0 for item in unique_items}
    # Set the value to 1 if the item was bought in the transaction
    for item in bought_together:
        binary_row[item] = 1
    # Append the binary row to the binary_data dictionary
    for item, value in binary_row.items():
        binary_data[item].append(value)
 
#Create a DataFrame from the dictionary
binary_df = pd.DataFrame(binary_data)
 
# Concatenate with the original DataFrame
binary_df2 = pd.concat([df1['Order_ID'], binary_df], axis=1)
 
# Display the binary DataFrame
print(binary_df)

       Instant food products  UHT-milk  abrasive cleaner  almonds  \
0                          0         0                 0        0   
1                          0         0                 0        0   
2                          0         0                 0        0   
3                          0         0                 0        0   
4                          0         0                 0        0   
...                      ...       ...               ...      ...   
12160                      0         0                 0        0   
12161                      0         0                 0        0   
12162                      0         0                 0        0   
12163                      0         0                 0        0   
12164                      0         1                 0        0   

       antioxydant juice  artif. sweetener  asparagus  avocado  babies food  \
0                      0                 0          0        0            0   
1            

In [27]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
# Load your dataset
frequent_itemsets = apriori(binary_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
# Filter rules based on desired metrics
#rules
filtered_rules = pd.DataFrame(rules[(rules['confidence'] > 0.01) & (rules['lift'] > 0.05)])
#
# # Print the filtered rules
print(filtered_rules)

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/tmp/pip_packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                      antecedents                   consequents  \
0                          (beef)            (other vegetables)   
1              (other vegetables)                        (beef)   
2                          (beef)                  (rolls/buns)   
3                    (rolls/buns)                        (beef)   
4                          (beef)             (root vegetables)   
..                            ...                           ...   
629          (yogurt, whole milk)              (tropical fruit)   
630  (tropical fruit, whole milk)                      (yogurt)   
631                      (yogurt)  (tropical fruit, whole milk)   
632              (tropical fruit)          (yogurt, whole milk)   
633                  (whole milk)      (yogurt, tropical fruit)   

     antecedent support  consequent support   support  confidence      lift  \
0              0.040855            0.150760  0.016441    0.402414  2.669232   
1              0.150760            0.

In [26]:
#pip install mlxtend

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
filtered_rules

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.040855,0.150760,0.016441,0.402414,2.669232,0.010281,1.421118,0.651998
1,(other vegetables),(beef),0.150760,0.040855,0.016441,0.109051,2.669232,0.010281,1.076544,0.736377
2,(beef),(rolls/buns),0.040855,0.140567,0.011508,0.281690,2.003954,0.005766,1.196465,0.522326
3,(rolls/buns),(beef),0.140567,0.040855,0.011508,0.081871,2.003954,0.005766,1.044674,0.582927
4,(beef),(root vegetables),0.040855,0.086067,0.014386,0.352113,4.091166,0.010869,1.410636,0.787755
...,...,...,...,...,...,...,...,...,...,...
629,"(yogurt, whole milk)",(tropical fruit),0.046691,0.082861,0.012906,0.276408,3.335822,0.009037,1.267482,0.734520
630,"(tropical fruit, whole milk)",(yogurt),0.035347,0.109248,0.012906,0.365116,3.342091,0.009044,1.403016,0.726465
631,(yogurt),"(tropical fruit, whole milk)",0.109248,0.035347,0.012906,0.118134,3.342091,0.009044,1.093877,0.786735
632,(tropical fruit),"(yogurt, whole milk)",0.082861,0.046691,0.012906,0.155754,3.335822,0.009037,1.129183,0.763487


In [14]:
import json
filtered_rules['antecedents'] = list(filtered_rules['antecedents'])

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
file = 'filtered_rules.csv'
filtered_rules.to_csv(file , index=False)

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
from refractio import snowflake
snowflake.execute_query(f"PUT file://{file} @FDC.INSIGHT_DESIGNER.INSIGHT_STAGE")

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Exception occurred in execute_query: Unknown error


In [31]:
pip install openpyxl

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     |████████████████████████████████| 256kB 19.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
filtered_rules = pd.DataFrame(filtered_rules)
filtered_rules

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.040855,0.150760,0.016441,0.402414,2.669232,0.010281,1.421118,0.651998
1,(other vegetables),(beef),0.150760,0.040855,0.016441,0.109051,2.669232,0.010281,1.076544,0.736377
2,(beef),(rolls/buns),0.040855,0.140567,0.011508,0.281690,2.003954,0.005766,1.196465,0.522326
3,(rolls/buns),(beef),0.140567,0.040855,0.011508,0.081871,2.003954,0.005766,1.044674,0.582927
4,(beef),(root vegetables),0.040855,0.086067,0.014386,0.352113,4.091166,0.010869,1.410636,0.787755
...,...,...,...,...,...,...,...,...,...,...
629,"(yogurt, whole milk)",(tropical fruit),0.046691,0.082861,0.012906,0.276408,3.335822,0.009037,1.267482,0.734520
630,"(tropical fruit, whole milk)",(yogurt),0.035347,0.109248,0.012906,0.365116,3.342091,0.009044,1.403016,0.726465
631,(yogurt),"(tropical fruit, whole milk)",0.109248,0.035347,0.012906,0.118134,3.342091,0.009044,1.093877,0.786735
632,(tropical fruit),"(yogurt, whole milk)",0.082861,0.046691,0.012906,0.155754,3.335822,0.009037,1.129183,0.763487


In [32]:
filtered_rules.to_excel('filtered_rules.xlsx')

/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
